# Experimenting new graph generative model evaluation metrics with GIN

Implementation of the whole GNN-based Evaluation workflow.
In short :

1. Initialization and Training of a graph classifier model using GIN.
1. Computing Graph embeddings using the trained models.
1. Computing evaluation metrics by computing distance metrics between embedddings of sets of graphs.

## Libraries Setup

In [1]:
import os

from models.training_GIN import train, test
%load_ext autoreload
%autoreload 2
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle
from tqdm import tqdm
from util import load_graph_list,perturb_new_rewire
from util import load_synth_data, separate_data , load_graph_asS2Vgraph
from models.graphcnn import GraphCNN
from GNN_metrics import compute_gnn_metrics

import pandas as pd
import time
from itertools import product
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import networkx as nx
%matplotlib inline
import seaborn as sns
criterion = nn.CrossEntropyLoss()

## Training a graph classifier model using Graph Isomorphism Network

### Hyperparameters
By default, we use an hyperparameter configuration already use with GIN.

In [2]:
dataset = None
device = 0
batch_size = 32
iters_per_epoch = 50
epochs = 60
lr = 0.01
seed = 0
fold_idx = 1
num_layers = [5]
num_mlp_layers = 2
hidden_dims = [64]
final_dropout = 0.5
graph_pooling_type = "sum"
neighbor_pooling_type = "sum"
learn_eps = False
degree_as_tag = True
filename = ""
random = 0
degree_feature= False
onehot_degree= True

### Initialize dataset for the graph classification task

The graph dataset consist in multiple classes corresponding to types of graphs generated from stocastic models :
1. Watts-Strogatz graphs
1. Barabasi-Albert graphs
1. 2-Community graphs
1. 2-Community (small) graphs
1. Ladder graphs
1. Grid graphs
1. 5 classes of ER graphs "equivalent" to the WS,BA,2-Com (Large and Small) and Grid graphs

In [3]:
row_list=[]

#set up seeds and gpu device
torch.manual_seed(0)
np.random.seed(0)
device = torch.device("cuda:" + str(device)) if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

graphs, num_classes, tagset , lentagset = load_synth_data(True, random, onehot_degree, True,degree_feature)

grid_graph=[]

for i in range(10,20):
    for j in range(10,20):
        grid_graph.append(nx.grid_2d_graph(i,j))

loading data
# classes: 11
# maximum node tag: 77
# data: 5100


### Training GIN

In [4]:
# fold_test_accuracy=[]
# fold_train_accuracy=[]
row_list=[]
for num_layer,hidden_dim in product(num_layers,hidden_dims):

    for h in range(1): # Training loops ( >1 if training multiple times )
        train_graphs, test_graphs = separate_data(graphs, seed, 1)

        model = GraphCNN(num_layer, num_mlp_layers, train_graphs[0].node_features.shape[1], hidden_dim, num_classes, final_dropout, learn_eps, graph_pooling_type, neighbor_pooling_type,random, device).to(device)

        optimizer = optim.Adam(model.parameters(),lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

        for epoch in range(1, epochs+1):
            scheduler.step()

            avg_loss = train(iters_per_epoch,batch_size,model, device, train_graphs, optimizer, criterion,epoch)
            acc_train, acc_test = test( model, device, train_graphs, test_graphs, epoch)
            # fold_test_accuracy.append(acc_test)
            # fold_train_accuracy.append(acc_train)
        # row_list.append(get_fid('../../generated_graphs/grid_GRANMixtureBernoulli_DFS.p',5,model))
        # row_list.append(get_fid('../../generated_graphs/grid_RNN_BFS.p',5,model))
        # row_list.append(get_fid('../../generated_graphs/grid_RNN_MLP_BFS.p',5,model))
        # for i in np.arange(0,1.1,0.1):
        #     row_list.append(get_fid_synth(perturb_new_rewire(ba_graph,i),1,model,str(i)))

    # train_acc.append((np.mean(fold_train_accuracy),num_layer,hidden_dim))
    # test_acc.append((np.mean(fold_test_accuracy),num_layer,hidden_dim))

D:\anaconda3\envs\graphRL\lib\site-packages\torch\optim\lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
epoch: 1: 100%|██████████| 50/50 [00:03<00:00, 13.32batch/s]


loss training: 41.839385


epoch: 2:   6%|▌         | 3/50 [00:00<00:01, 27.60batch/s]

accuracy train: 0.665359 test: 0.668627


epoch: 2: 100%|██████████| 50/50 [00:01<00:00, 28.86batch/s]


...

loss training: 0.236195


epoch: 60:   6%|▌         | 3/50 [00:00<00:01, 26.86batch/s]

accuracy train: 0.999564 test: 1.000000


epoch: 60: 100%|██████████| 50/50 [00:01<00:00, 28.09batch/s]


loss training: 0.164309
accuracy train: 0.999782 test: 1.000000


### Computing Graph embeddings using the trained GIN model


In [9]:
def get_gnn_metrics(filename,graph_dataset,label,model):
    """
    Get the gnn-based metrics (FD,PRDC), comparing the graphs from the file data with
    with the reference one used as graphs during GIN training
    :param filename: string
    :param graph_dataset: set of networkx graphs
    :param label: the corresponding integer label for the given class of graphs (eg. if measuring Watts Strogatz = 1)
    :param model: the trained instance of the classifier model
    :return:
    """
    graph_gen = load_graph_list(filename,False)
    g_list,_=load_graph_asS2Vgraph(graph_gen,label,random,tagset,lentagset,onehot=onehot_degree,degree_feature=degree_feature)
    fid,prdc=compute_gnn_metrics([graph_dataset[i] for i in range(len(graph_dataset)) if graph_dataset[i].label ==label],g_list,model)
    print(filename+" fid : ",fid)
    print(filename+" prdc : ", prdc)

    return {"filename":filename,"FD":fid,"PRDC":prdc}

def get_gnn_metrics_synth(graph,graph_dataset,label,model,name):
    """
    Get the gnn-based metrics (FD,PRDC), comparing the graphs from the given with
    with the reference one used as graphs during GIN training
    :param graph: set of networkx graphs
    :param graph_dataset: set of networkx graphs
    :param label: the corresponding integer label for the given class of graphs (eg. if measuring Watts Strogatz = 1)
    :param model: the trained instance of the classifier model
    :param name: a string to name the given measured set of graphs (eg. "Grid", "GraphRNN-DFS Grid", etc)
    :return:
    """
    g_list,_=load_graph_asS2Vgraph(graph,label,random,tagset,lentagset,onehot=onehot_degree,degree_feature=degree_feature)
    fid,prdc=compute_gnn_metrics([graph_dataset[i] for i in range(len(graph_dataset)) if graph_dataset[i].label ==label],g_list,model)
    print(" fid : ",fid)
    print(" prdc : ", prdc)

    return {"filename":name,"FD":fid,"PRDC":prdc}

### Using computed graphs embeddings for experiments

Experiments :
1. Measuring GNN-based metrics for an increasingly dissimilar set of graphs ( by randomly rewiring edges of ones graphs)
2. Measuring GNN-based metrics for sets of graphs from several epochs of a trained generative graphs model ( here, GRAN on grid graphs)
3. Comparing GraphRNN and GRAN generated sets of graphs compared to a reference set of graphs (Grid experiment)

### 1) Perturbation Experiment
Perturbing a graph using random edge rewiring (at an increasing ratio from 0=base graph, to 1=random graph)

In [11]:
# Generating a new set of grid graph ( from the same data distribution
# as the one from the GNN classifier training dataset )
grid_graph=[]
for i in range(10,20):
    for j in range(10,20):
        grid_graph.append(nx.grid_2d_graph(i,j))

perturb_results = [] # Initializing list of gnn metrics results for each sets of perturbed graphs
for i in np.arange(0,1.1,0.1):
    perturb_results.append(get_gnn_metrics_synth(perturb_new_rewire(grid_graph,i),graphs,5,model,str(i)))
perturb_results_df = pd.DataFrame(perturb_results)
perturb_results_df

# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
 fid :  -0.00011094785440945998
 prdc :  {'precision': 1.0, 'recall': 1.0, 'density': 0.9800000000000001, 'coverage': 1.0}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
 fid :  19258.326319264503
 prdc :  {'precision': 0.0, 'recall': 0.0, 'density': 0.0, 'coverage': 0.0}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
 fid :  47343.591335743076
 prdc :  {'precision': 0.0, 'recall': 0.0, 'density': 0.0, 'coverage': 0.0}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
 fid :  63303.994141755094
 prdc :  {'precision': 0.0, 'recall': 0.0, 'density': 0.0, 'coverage': 0.0}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
 fid :  79244.87520859012
 prdc :  {'precision': 0.0, 'recall': 0.0, 'density': 0.0, 'coverage': 0.0}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
 fid :  97

,filename,FD,PRDC
0,0.0,-0.000111,"{'precision': 1.0, 'recall': 1.0, 'density': 0..."
1,0.1,19258.326319,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."
2,0.2,47343.591336,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."
3,0.30000000000000004,63303.994142,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."
4,0.4,79244.875209,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."
5,0.5,97988.897495,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."
6,0.6000000000000001,124504.544562,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."
7,0.7000000000000001,162644.000547,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."
8,0.8,216473.868366,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."
9,0.9,277162.532718,"{'precision': 0.0, 'recall': 0.0, 'density': 0..."


### 2) Model Selection Experiment
Taking several set of graphs from several epochs of the training phase of one generative model (eg GRAN on Grid graphs)

In [20]:
grid_graph=[]
for i in range(10,20):
    for j in range(10,20):
        grid_graph.append(nx.grid_2d_graph(i,j))

comparison_results = []

dir = os.fsencode("grid_graph_gran")

for file in os.listdir(dir):
    filename=os.fsdecode(file)
    comparison_results.append(get_gnn_metrics("grid_graph_gran\\"+filename,graphs,5,model))
comparison_results_df = pd.DataFrame(comparison_results)
comparison_results_df

# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
grid_graph_gran\grid_GRANMixtureBernoulli_DFS_0000300.p fid :  29926.133367172246
grid_graph_gran\grid_GRANMixtureBernoulli_DFS_0000300.p prdc :  {'precision': 0.34, 'recall': 1.0, 'density': 0.35800000000000004, 'coverage': 0.67}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
grid_graph_gran\grid_GRANMixtureBernoulli_DFS_0000600.p fid :  14765.325564828256
grid_graph_gran\grid_GRANMixtureBernoulli_DFS_0000600.p prdc :  {'precision': 0.33, 'recall': 1.0, 'density': 0.30000000000000004, 'coverage': 0.77}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
grid_graph_gran\grid_GRANMixtureBernoulli_DFS_0000900.p fid :  5368.885847922869
grid_graph_gran\grid_GRANMixtureBernoulli_DFS_0000900.p prdc :  {'precision': 0.73, 'recall': 1.0, 'density': 0.738, 'coverage': 0.94}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
grid_graph_gran\gr

,filename,FD,PRDC
0,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,29926.133367,"{'precision': 0.34, 'recall': 1.0, 'density': ..."
1,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,14765.325565,"{'precision': 0.33, 'recall': 1.0, 'density': ..."
2,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,5368.885848,"{'precision': 0.73, 'recall': 1.0, 'density': ..."
3,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,135824.796179,"{'precision': 0.02, 'recall': 1.0, 'density': ..."
4,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,798.441929,"{'precision': 0.74, 'recall': 0.99, 'density':..."
5,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,1726.274521,"{'precision': 0.79, 'recall': 1.0, 'density': ..."
6,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,1856.225555,"{'precision': 0.74, 'recall': 1.0, 'density': ..."
7,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,3228.604082,"{'precision': 0.75, 'recall': 1.0, 'density': ..."
8,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,3852.612152,"{'precision': 0.8, 'recall': 1.0, 'density': 0..."
9,grid_graph_gran\grid_GRANMixtureBernoulli_DFS_...,323.907076,"{'precision': 0.83, 'recall': 1.0, 'density': ..."


### 3) Model Comparison Experiment (Grid experiment)
Taking several set of graphs from generative models (here, GRAN vs GraphRNN vs GraphRNN-S on Grid)

In [19]:
grid_graph=[]
for i in range(10,20):
    for j in range(10,20):
        grid_graph.append(nx.grid_2d_graph(i,j))

comparison_results = []

dir = os.fsencode("saved_graphs")

for file in os.listdir(dir):
    filename=os.fsdecode(file)
    comparison_results.append(get_gnn_metrics("saved_graphs\\"+filename,graphs,5,model))
comparison_results_df = pd.DataFrame(comparison_results)
comparison_results_df

# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
saved_graphs\grid_GRANMixtureBernoulli_DFS.p fid :  323.90707624417155
saved_graphs\grid_GRANMixtureBernoulli_DFS.p prdc :  {'precision': 0.83, 'recall': 1.0, 'density': 0.8180000000000001, 'coverage': 0.96}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
saved_graphs\grid_RNN_BFS.p fid :  37102.80277746646
saved_graphs\grid_RNN_BFS.p prdc :  {'precision': 0.11, 'recall': 0.03, 'density': 0.034, 'coverage': 0.03}
# classes: 1
# maximum node tag: 77
# data: 100
Num real: 100 Num fake: 100
saved_graphs\grid_RNN_MLP_BFS.p fid :  53662.08717673054
saved_graphs\grid_RNN_MLP_BFS.p prdc :  {'precision': 0.0, 'recall': 0.41, 'density': 0.0, 'coverage': 0.0}


,filename,FD,PRDC
0,saved_graphs\grid_GRANMixtureBernoulli_DFS.p,323.907076,"{'precision': 0.83, 'recall': 1.0, 'density': ..."
1,saved_graphs\grid_RNN_BFS.p,37102.802777,"{'precision': 0.11, 'recall': 0.03, 'density':..."
2,saved_graphs\grid_RNN_MLP_BFS.p,53662.087177,"{'precision': 0.0, 'recall': 0.41, 'density': ..."
